In [1]:
import numpy as np
from numpy import genfromtxt

preprocessed_data = genfromtxt('data/preprocessed_data.csv', delimiter=',', dtype = str)

In [2]:
summary_list = []

for row in preprocessed_data:
    if row[10] not in summary_list:
        summary_list.append(row[10])

print(len(summary_list))
print(summary_list)

34
['"""Clear"""', '"""Partly Cloudy"""', '"""Overcast"""', '"""Humid and Mostly Cloudy"""', '"""Humid and Overcast"""', '"""Humid and Partly Cloudy"""', '"""Humid"""', '"""Mostly Cloudy"""', '"""Rain"""', '"""Heavy Rain"""', '"""Possible Light Rain"""', '"""Foggy"""', '"""Light Rain"""', '"""Breezy"""', '"""Breezy and Partly Cloudy"""', '"""Breezy and Mostly Cloudy"""', '"""Drizzle"""', '"""Heavy Rain and Breezy"""', '"""Rain and Breezy"""', '"""Light Rain and Breezy"""', '"""Heavy Rain and Windy"""', '"""Possible Drizzle"""', '"""Humid and Foggy"""', '"""Dry"""', '"""Breezy and Overcast"""', '"""Possible Light Rain and Humid"""', '"""Possible Drizzle and Humid"""', '"""Rain and Humid"""', '"""Windy and Mostly Cloudy"""', '"""Light Rain and Humid"""', '"""Drizzle and Humid"""', '"""Heavy Rain and Humid"""', '"""Possible Light Sleet"""', '"""Light Sleet"""']


In [3]:
summary_enum = dict()
count = 0
for s in summary_list:
    summary_enum[s] = count
    count += 1


In [4]:
print(len(preprocessed_data[0]))

21


In [51]:
#x_indices = [5, 6, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
#y_indices = [7]
x_indices = [3, 11]
y_indices = [5]

In [52]:
x = []
y = []

for row in preprocessed_data:
    temp_x_list = []
    temp_y_list = []
    for i in range(len(row)):
        if i in x_indices:
            temp_x_list.append(float(row[i]))
        if i in y_indices:
            temp_y_list.append(float(row[i]))
    x.append(temp_x_list)
    y.append(temp_y_list)

In [53]:
x_ray = np.array(x)
y_ray = np.array(y)

print(x_ray.shape)
print(y_ray.shape)

(922747, 2)
(922747, 1)


In [54]:
from sklearn.model_selection import train_test_split

In [55]:
X_train, X_test, y_train, y_test = train_test_split(x_ray, y_ray)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(692060, 2)
(230687, 2)
(692060, 1)
(230687, 1)


In [56]:
def ridge(x, y, lam):
    """This function does ridge regression with the stimuli x and responses y with
    ridge parameter lam (short for lambda). It returns the weights.
    This is definitely not the most efficient way to do this, but it's fine for now.
    """
    n_features = x.shape[1]
    beta_ridge = np.linalg.inv(x.T.dot(x) + lam * np.eye(n_features)).dot(x.T).dot(y)
    return beta_ridge

In [57]:
def mean_squared_error(z, z_hat):
    sum = 0.0
    for i in range(len(z)):
        sum += (z[i] - z_hat[i]) ** 2
    return sum / len(z)

In [58]:
import matplotlib.pyplot as plt

In [ ]:
n_training, n_features = X_train.shape

n_mc_iters = 50 # let's do 50 Monte Carlo iterations
n_per_mc_iter = 50 # on each MC iteration, hold out 50 datapoints to be the validation set

lambdas = np.logspace(-3, 10, 30)
val_mses = np.zeros((n_training, len(lambdas)))

indices = np.arange(X_train.shape[0])

for it in range(n_mc_iters):
    # split the training dataset into two parts: one with a random selection of n_per_mc_iter points
    # and one with all the other datapoints
    
    subindices = np.random.choice(indices, n_per_mc_iter, replace = False)
    
    x_trn = np.delete(X_train, subindices, 0)
    y_trn = np.delete(y_train, subindices, 0)
    
    
    x_val = np.take(X_train, subindices, 0)
    y_val = np.take(y_train, subindices, 0)
    
    for ii in range(len(lambdas)):
        # fit model using x_trn & predict y_val
        # predict y_val
        y_val_hat = x_val.dot(ridge(x_trn, y_trn, lambdas[ii]))
        
        # store squared error in val_mses
        val_mses[it,ii] = mean_squared_error(y_val, y_val_hat)



In [ ]:
# Plot the MSE for each lambda, averaged across the MC iterations

plt.figure()
plt.plot(np.sum(val_mses, axis = 0)/n_mc_iters)

In [ ]:
# Choose the best lambda, print it
best_lambda = np.argmin(np.sum(val_mses, axis = 0)/n_mc_iters)
print("Best lambda:", best_lambda)

# Fit a model using the whole training set and the best lambda
beta_hat = ridge(X_train, y_train, lambdas[best_lambda])

# Use that model to predict y_test
y_test_hat = X_test.dot(beta_hat)

# Compute the MSE, print it
test_mse = mean_squared_error(y_test, y_test_hat)

print("Test MSE:", test_mse)

In [ ]:
print(beta_hat)